#지역별 WIFI 설치 현황 분석 및 무료 WIFI 지도 시각화

In [1]:
#한글깨짐 방지 패키지 설치
!pip install koreanize-matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


라이브러리 임포트

In [2]:
import pandas as pd

import plotly.express as px
import koreanize_matplotlib

import folium

데이터 준비하기

In [3]:
df = pd.read_csv('/content/2020년 지역별 WIFI 설치현황.csv', encoding = 'EUC-KR')
df.head()

,설치시도명,설치시군구명,설치시설구분,서비스제공사명,설치년월,관리기관명,위도,경도,데이터기준일자
0,제주특별자치도,서귀포시,편의시설,KT,NaN,정보정책과,33.251523,126.559351,2020-10-28
1,제주특별자치도,서귀포시,편의시설,KT,NaN,정보정책과,33.251523,126.559351,2020-10-28
2,제주특별자치도,서귀포시,교통시설,KT,NaN,정보정책과,33.251395,126.559516,2020-10-28
3,제주특별자치도,서귀포시,교통시설,KT,NaN,정보정책과,33.251395,126.559516,2020-10-28
4,제주특별자치도,서귀포시,관광,KT,NaN,정보정책과,33.249872,126.559884,2020-10-28


데이터 확인하기

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25669 entries, 0 to 25668
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   설치시도명    25669 non-null  object 
 1   설치시군구명   25669 non-null  object 
 2   설치시설구분   25669 non-null  object 
 3   서비스제공사명  25669 non-null  object 
 4   설치년월     20252 non-null  object 
 5   관리기관명    25669 non-null  object 
 6   위도       25669 non-null  float64
 7   경도       25669 non-null  float64
 8   데이터기준일자  25669 non-null  object 
dtypes: float64(2), object(7)
memory usage: 1.8+ MB


In [5]:
#전체 데이터의 결측치 확인
df.isna().sum()

설치시도명         0
설치시군구명        0
설치시설구분        0
서비스제공사명       0
설치년월       5417
관리기관명         0
위도            0
경도            0
데이터기준일자       0
dtype: int64

In [6]:
df['설치시설구분'].unique() # unique() => 중복을 제외한 값을 array로 리턴 해줌
#df.설치시설구분

array(['편의시설', '교통시설', '관광', '주요거리', '공원', '공공기관', '복지센터등', '관공서',
       '서민·복지시설', '기타', '문화관광', '전통시장', '버스승차대', '지역문화시설', '서민/복지시설',
       '교육시설', '공공도서관', '공원내', '민원실', '쉼터', '휴게실', '박물관', '시장', '도서관',
       '노인복지회관', '행정복지센터', '다중이용시설', '버스정류장', '관광지역', '관광지', '복지시설',
       '보건지소', '정류장', '버스', '관광시설', '보건진료소', '서민복지시설', '터미널', '종합복지시설',
       '읍/면/동 주민센터', ' 서민·복지시설', '보건소', '공공(일반)', '주민자치센터', '노인정',
       '문화체육시설', '버스터미널'], dtype=object)

In [7]:
len(df['설치시설구분'].unique())

47

데이터 전처리

In [8]:
df['설치시설구분'].value_counts() #서민·복지시설, 서민/복지시설,  서민·복지시설, 서민복지시설

관공서           8782
서민·복지시설       3657
기타            3063
교통시설          2787
관광            2300
편의시설          2143
지역문화시설        1817
교육시설           321
공원             123
버스정류장          112
주요거리           101
서민/복지시설         67
관광지             62
다중이용시설          41
복지센터등           41
버스              37
공공기관            33
문화관광            32
전통시장            27
정류장             16
보건지소            14
행정복지센터          13
보건소             10
민원실              9
시장               7
보건진료소            6
관광시설             5
복지시설             5
노인정              4
공원내              4
버스승차대            4
공공도서관            4
문화체육시설           3
주민자치센터           3
공공(일반)           2
휴게실              2
서민복지시설           2
읍/면/동 주민센터       1
 서민·복지시설         1
도서관              1
종합복지시설           1
터미널              1
관광지역             1
노인복지회관           1
박물관              1
쉼터               1
버스터미널            1
Name: 설치시설구분, dtype: int64

In [9]:
#서민·복지시설, 서민/복지시설,  서민·복지시설, 서민복지시설 => 서민·복지시설(전처리 하기)
# 서민 키워드가 포함된 설치시설구분 -> 서민·복지시설(전처리 하기)
df.loc[df['설치시설구분'].str.contains('서민'), '설치시설구분'] = '서민·복지시설'

df['설치시설구분'].unique()


array(['편의시설', '교통시설', '관광', '주요거리', '공원', '공공기관', '복지센터등', '관공서',
       '서민·복지시설', '기타', '문화관광', '전통시장', '버스승차대', '지역문화시설', '교육시설',
       '공공도서관', '공원내', '민원실', '쉼터', '휴게실', '박물관', '시장', '도서관', '노인복지회관',
       '행정복지센터', '다중이용시설', '버스정류장', '관광지역', '관광지', '복지시설', '보건지소', '정류장',
       '버스', '관광시설', '보건진료소', '터미널', '종합복지시설', '읍/면/동 주민센터', '보건소',
       '공공(일반)', '주민자치센터', '노인정', '문화체육시설', '버스터미널'], dtype=object)

In [10]:
df.loc[df['설치시설구분'].str.contains('서민'), '설치시설구분'].unique()

array(['서민·복지시설'], dtype=object)

In [11]:
df['설치시설구분'].value_counts()

관공서           8782
서민·복지시설       3727
기타            3063
교통시설          2787
관광            2300
편의시설          2143
지역문화시설        1817
교육시설           321
공원             123
버스정류장          112
주요거리           101
관광지             62
복지센터등           41
다중이용시설          41
버스              37
공공기관            33
문화관광            32
전통시장            27
정류장             16
보건지소            14
행정복지센터          13
보건소             10
민원실              9
시장               7
보건진료소            6
복지시설             5
관광시설             5
노인정              4
공원내              4
공공도서관            4
버스승차대            4
주민자치센터           3
문화체육시설           3
휴게실              2
공공(일반)           2
읍/면/동 주민센터       1
도서관              1
종합복지시설           1
터미널              1
관광지역             1
노인복지회관           1
박물관              1
쉼터               1
버스터미널            1
Name: 설치시설구분, dtype: int64

지역별 WIFI 설치 현황 데이터 분석

In [13]:
# 설치시도별 무료 WIFI 설치 현황 수
df['설치시도명'].unique()
df['설치시도명'].value_counts()

서울특별시      5185
경기도        2965
전라남도       2057
강원도        2038
인천광역시      1714
경상북도       1694
부산광역시      1395
전라북도       1339
충청남도       1298
광주광역시      1247
경상남도       1236
충청북도       1023
대구광역시       806
울산광역시       649
제주특별자치도     500
대전광역시       302
세종특별자치시     221
Name: 설치시도명, dtype: int64

In [16]:
# 설치시도별 설치시설구분과 서비스제공사의 설치 현황의 집계표
group_df = df.groupby(by = ['설치시도명', '설치시설구분', '서비스제공사명']).size().reset_index(name = '설치수') # count를 하게 되면 확인이 가능함, size는 수를 줌
group_df

,설치시도명,설치시설구분,서비스제공사명,설치수
0,강원도,관공서,(주)엘지유플러스,7
1,강원도,관공서,KT,134
2,강원도,관공서,LG U+,40
3,강원도,관공서,LGU+,82
4,강원도,관공서,SKT,107
...,...,...,...,...
956,충청북도,편의시설,KT,22
957,충청북도,편의시설,SKT,15
958,충청북도,편의시설,보은군청,3
959,충청북도,편의시설,충주시,1


In [17]:
#서비스제공사별 WIFI 설치 현황
px.histogram(group_df, x = '서비스제공사명', y = '설치수')

In [19]:
#설치시설구분 설치 현황
px.histogram(group_df, x = '설치시설구분', y = '설치수')

In [21]:
#설치시도명 설치 현황
px.histogram(group_df, x = '설치시도명', y = '설치수')

In [22]:
px.bar(group_df, x = '설치시도명', y = '설치수')

In [23]:
# 설치시도별 WIFI 설치 비율
fig = px.pie(group_df, names = '설치시도명', values = '설치수')
fig.show()

In [24]:
# 설치시도별 WIFI 설치 비율
fig = px.pie(group_df, names = '설치시도명', values = '설치수')
fig.update_traces(textposition = 'inside', textinfo = 'percent + label')
fig.show()

In [27]:
# 설치시도별 WIFI 설치 비율
colors = ['gold', 'mediumturquoise', 'darkorange', 'lightgreen']
fig = px.pie(group_df, names = '설치시도명', values = '설치수')
fig.update_traces(textposition = 'inside', textinfo = 'percent + label', hoverinfo = 'value', textfont_size = 20,
                  marker = dict(colors = colors, line = dict(color = '#000000', width = 2)))
fig.show()

In [28]:
# 설치시설구분 WIFI 설치 비율
fig = px.pie(group_df, names = '설치시설구분', values = '설치수')
fig.update_traces(textposition = 'inside', textinfo = 'percent + label')
fig.show()

서울특별시 무료 WIFI 현황 데이터 분석 및 시각화

In [30]:
sl_df = df.loc[df['설치시도명'] == '서울특별시']
sl_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5185 entries, 27 to 25360
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   설치시도명    5185 non-null   object 
 1   설치시군구명   5185 non-null   object 
 2   설치시설구분   5185 non-null   object 
 3   서비스제공사명  5185 non-null   object 
 4   설치년월     3979 non-null   object 
 5   관리기관명    5185 non-null   object 
 6   위도       5185 non-null   float64
 7   경도       5185 non-null   float64
 8   데이터기준일자  5185 non-null   object 
dtypes: float64(2), object(7)
memory usage: 405.1+ KB


In [31]:
# 서울특별시(구별)는 어떤 목적으로 무료 WIFI를 설치했나?
# 구로구 -> 기타 왜 기타만 있나 확인 필요
# 성동구 지역문화시설 많음 -> 확인 필요
# 광진구 관광 -> 지도 시각화 확인 필요
# 서대문구 주요거리 -> 확인 필요
# 강남구 편의시설 많음 -> 확인 필요
# 설치시설구분으로 마커를 다르게 표출하기
fig = px.histogram(sl_df, x = '설치시군구명', color = '설치시설구분')
fig.show()

In [32]:
# 어떤 관리기관들이 무료 WIFI를 관리하고 있나?
# 서대문구, 광진구, 서울시에서 관리하고 있는 무료 WIFI는 무엇이지?
fig = px.histogram(sl_df, x = '설치시군구명', color = '관리기관명')
fig.show()

In [33]:
# 서울특별시 구별 WIFI 설치 비율

# 설치시군구별 WIFI 설치 수 집계표
sl_group_df = sl_df.groupby(by = ['설치시군구명', '설치시설구분', '서비스제공사명', '관리기관명']).size().reset_index(name = '설치수')

fig = px.pie(sl_group_df, names = '설치시군구명', values = '설치수')
fig.update_traces(textposition = 'inside', textinfo = 'percent + label')
fig.show()

In [34]:
# 시군구별 WIFI 설치수와 
fig = px.histogram(sl_group_df, x = '설치시군구명', y = '설치수', color = '설치시설구분')
fig.show()

In [35]:
# 서울시 구의 WIFI 설치 서비스제공사별 설치수
fig = px.histogram(sl_group_df, x = '설치시군구명', y = '설치수', color = '서비스제공사명')
fig.show()

내위치와 가장 가까운 무료 WIFI 찾아서 지도 시각화 서비스

In [40]:
# 1) 내 위치 좌표 찾기
# 1-1) folium 지도에 click 이벤트를 주면 좌표정보가 popup 으로 띄어주고 그걸 확인해서 내 위치좌표로 사용한다.

# 지도 생성
m = folium.Map(location = [36.3536, 127.4231], zoom_start = 13)

# clickForMarker 함수를 적용 -> click 이벤트를 주면 마커를 등록
m.add_child(folium.ClickForMarker(popup = 'point'))

# 위도/경도 팝업 활성화 시키기 -> 지도를 탐색하여 좌표값 찾는데 활용
m.add_child(folium.LatLngPopup())


In [ ]:
# 1-2) 도로명 주소를 입력하면 -> 좌표값으로 변환
from geopy.geocoders import Nominatim

# 도로명 주소 입력하면 좌표값을 반환하는 함수를 선언
def geocoding(address):
  geocoder = Nominatim(user_agent= 'South Korea', timeout = None)
  geo = geocoder.geocode(address)
  crd = {"lat" : float(geo.latitude), "lng" : float(geo.longitude)}

  return crd

address = input('도로명 주소를 입력하세요 : ')
crd = geocoding(address)
print(crd['lat'])
print(crd['lng'])

# 지도에 내 위치 마커 표시
my_home = (crd['lat'],crd['lng']) # 거리르 계산하려면 튜플로 해야햐기 때문에 튜플로 지정
my_map = folium.Map(location = [crd['lat'], crd['lng']], zoom_start = 13)

folium.Marker([crd['lat'],crd['lng']], icon =folium.Icon(color = 'red', icon = 'fa-home', prefix = 'fa')).add_to(my_map)
my_map


In [44]:
# 2) 내 위치랑 가장 가까운 WIFI TOP 10 찾기
# 모든 WIFI 위치 좌표 - 내 위치 자표 => 가장 거리가 적은(가까운) TOP10
# 좌표 거리 계산 필요
from geopy.distance import geodesic

# 지도시각화에 필요한 정보만 담은 df
my_wifi = pd.DataFrame(columns= ['설치시군구명', '설치시설구분', '위도', '경도', '거리'])

for n in df.index:
  # 거리 계산을 위해 위도경도 튜플 자료구조로 변환
  wifi_loc = (df.loc[n,'위도'], df.loc[n,'경도'])
  # my_wifi 데이터프레임에 값 담기
  my_wifi.loc[n] = [df.loc[n,'설치시군구명'], df.loc[n,'설치시설구분'], df.loc[n,'위도'], df.loc[n,'경도'],
                    geodesic(my_home, wifi_loc).kilometers]


my_wifi.head()


,설치시군구명,설치시설구분,위도,경도,거리
0,서귀포시,편의시설,33.251523,126.559351,346.286963
1,서귀포시,편의시설,33.251523,126.559351,346.286963
2,서귀포시,교통시설,33.251395,126.559516,346.297813
3,서귀포시,교통시설,33.251395,126.559516,346.297813
4,서귀포시,관광,33.249872,126.559884,346.456385


In [45]:
# 내 위치와 가장 가까운 무료 WIFI TOP 10
my_wifi_top10 = my_wifi.sort_values(by = ['거리']).head(10)
my_wifi_top10

,설치시군구명,설치시설구분,위도,경도,거리
17284,서구,서민·복지시설,36.306143,127.337337,0.035701
18573,서구,관공서,36.303857,127.337354,0.281034
15130,서구,서민·복지시설,36.309976,127.339148,0.440757
15132,서구,서민·복지시설,36.302452,127.338390,0.451673
15131,서구,서민·복지시설,36.302452,127.338390,0.451673
12622,서구,서민·복지시설,36.305316,127.345190,0.738944
18574,서구,관공서,36.299285,127.335192,0.804998
18575,서구,관공서,36.301099,127.345011,0.923153
18569,서구,관공서,36.303549,127.349368,1.148543
18576,서구,서민·복지시설,36.298335,127.325373,1.378382


In [49]:
# 3) 지도 시각화
my_map = folium.Map(location = [crd['lat'], crd['lng']], zoom_start = 13)

for n in my_wifi_top10.index:
  folium.Marker([my_wifi_top10.loc[n, '위도'],my_wifi_top10.loc[n,'경도']],
                popup = '<pre>' + my_wifi_top10.loc[n, '설치시설구분'] + '</pre>',
                icon =folium.Icon(icon = 'wifi', prefix = 'fa')).add_to(my_map)


folium.Marker([crd['lat'],crd['lng']], icon =folium.Icon(color = 'red', icon = 'fa-home', prefix = 'fa')).add_to(my_map)
my_map


사용자에게 도로명 주소를 입력받아 마커를 표시하고 가장 가까운 무료 WIFI 10곳을 지도 시각화 한다. 단 하나의 셀에서 실행 되어야 함

In [50]:
# 1) 내 위치 좌표 찾기
from geopy.geocoders import Nominatim

m = folium.Map(location = [36.3536, 127.4231], zoom_start = 13)
m.add_child(folium.ClickForMarker(popup = 'point'))
m.add_child(folium.LatLngPopup())

# 2) 도로명 주소를 입력하면 -> 좌표값으로 변환
# 도로명 주소 입력하면 좌표값을 반환하는 함수를 선언

def geocoding(address):
  geocoder = Nominatim(user_agent= 'South Korea', timeout = None)
  geo = geocoder.geocode(address)
  crd = {"lat" : float(geo.latitude), "lng" : float(geo.longitude)}

  return crd

address = input('도로명 주소를 입력하세요 : ')
crd = geocoding(address)

my_home = (crd['lat'],crd['lng']) 
my_map = folium.Map(location = [crd['lat'], crd['lng']], zoom_start = 13)

folium.Marker([crd['lat'],crd['lng']], icon =folium.Icon(color = 'red', icon = 'fa-home', prefix = 'fa')).add_to(my_map)
from geopy.distance import geodesic

# 3) 내 위치랑 가장 가까운 WIFI TOP 10 찾기
my_wifi = pd.DataFrame(columns= ['설치시군구명', '설치시설구분', '위도', '경도', '거리'])

for n in df.index:
  wifi_loc = (df.loc[n,'위도'], df.loc[n,'경도'])
  my_wifi.loc[n] = [df.loc[n,'설치시군구명'], df.loc[n,'설치시설구분'], df.loc[n,'위도'], df.loc[n,'경도'],
                    geodesic(my_home, wifi_loc).kilometers]

my_wifi_top10 = my_wifi.sort_values(by = ['거리']).head(10)

# 4) 내 위치와 가장 가까운 무료 WIFI TOP 10
my_map = folium.Map(location = [crd['lat'], crd['lng']], zoom_start = 13)


# 5) 지도 시각화

for n in my_wifi_top10.index:
  folium.Marker([my_wifi_top10.loc[n, '위도'],my_wifi_top10.loc[n,'경도']],
                popup = '<pre>' + my_wifi_top10.loc[n, '설치시설구분'] + '</pre>',
                icon =folium.Icon(icon = 'wifi', prefix = 'fa')).add_to(my_map)


folium.Marker([crd['lat'],crd['lng']], icon =folium.Icon(color = 'red', icon = 'fa-home', prefix = 'fa')).add_to(my_map)
my_map



도로명 주소를 입력하세요 : 대전광역시 서구 관저북로 14
